In [1]:
import os
import requests
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
from datetime import datetime

In [2]:
BASE_URL = 'http://storage.googleapis.com/'

In [ ]:
# Key_json is a .json file follow instructions here: 
# https://www.stitchdata.com/docs/destinations/google-bigquery/v2/connecting-google-bigquery-to-stitch
# Pay attention only for STEP-1. You dont need to execute STEP-2
# create new project at google cloud platform

In [3]:
def query_sentinel(key_json, project_id, start, end, tile, cloud=100.):
    credentials = service_account.Credentials.from_service_account_file(key_json)
    client = bigquery.Client(credentials=credentials, project=project_id)
    query = client.query("""
                SELECT * FROM `bigquery-public-data.cloud_storage_geo_index.sentinel_2_index` 
                    WHERE mgrs_tile IN ("{t}") 
                    AND DATE(sensing_time) BETWEEN DATE("{s}") AND DATE("{e}")
                """.format(t=tile, s=start, e=end))
    results = query.result()
    df = results.to_dataframe()
    good_scenes = []
    for i, row in df.iterrows():
        print (row['product_id'], '; cloud cover:', row['cloud_cover'])
        if float(row['cloud_cover']) <= cloud:
            good_scenes.append(row['base_url'].replace('gs://', BASE_URL))
    return good_scenes

In [4]:
def download_file(url, dst_name):
    try:
        data = requests.get(url, stream=True)
        with open(dst_name, 'wb') as out_file:
            for chunk in data.iter_content(chunk_size=100 * 100):
                out_file.write(chunk)
    except:
        print ('\t ... {f} FAILED!'.format(f=url.split('/')[-1]))
    return

In [5]:
def make_safe_dirs(scene, outpath):
    scene_name = os.path.basename(scene)
    scene_path = os.path.join(outpath, scene_name)
    manifest = os.path.join(scene_path, 'manifest.safe')
    manifest_url = scene + '/manifest.safe'
    if os.path.exists(manifest):
        os.remove(manifest)
    download_file(manifest_url, manifest)
    with open(manifest, 'r') as f:
        manifest_lines = f.read().split()
    download_links = []
    load_this = False
    for line in manifest_lines:
        if 'href' in line:
            online_path = line[7:line.find('><') - 2]
            tile = scene_name.split('_')[-2]
            if online_path.startswith('/GRANULE/'):
                if '_' + tile + '_' in online_path:
                    load_this = True
            else:
                load_this = True
            if load_this:
                local_path = os.path.join(scene_path, *online_path.split('/')[1:])
                online_path = scene + online_path
                download_links.append((online_path, local_path))
        load_this = False
    for extra_dir in ('AUX_DATA', 'HTML'):
        if not os.path.exists(os.path.join(scene_path, extra_dir)):
            os.makedirs(os.path.join(scene_path, extra_dir))
    return download_links

In [6]:
def download_sentinel(scene, dst):
    scene_name = scene.split('/')[-1]
    scene_path = os.path.join(dst, scene_name)
    if not os.path.exists(scene_path):
        os.mkdir(scene_path)
    print ('Downloading scene {s} ...'.format(s=scene_name))
    download_links = sorted(make_safe_dirs(scene, dst))
    for l in download_links:
        if not os.path.exists(os.path.dirname(l[1])):
            os.makedirs(os.path.dirname(l[1]))
        if os.path.exists(l[1]):
            os.remove(l[1])
        if l[1].endswith('.jp2'):
            print ('\t ... *{b}'.format(b=l[1].split('_')[-1]))
        if download_file(l[0], l[1]) is False:
            print ('\t ... {f} failed to download! Download for this scene is cancelled here!'.format(f=l[0]))
            return

In [ ]:
# Read excel with tile and ground truth information
# The ground truth data have 13 columns namely: Area, Location, Quantity, Sentinel_pixels, Lat, Lon, tile, EventDate,
# Product_title, Image_date, CloudCover, id and DaysDiff
# In this script the 'Image_date' and 'tile' columns will be used.
# Set path of ground truth .csv
csv_path = r"/home/antonis/sentinel-2/sentinel2_products.csv"
df = pd.read_csv(csv_path, index_col=0, header=0,low_memory=False)

In [9]:
df = df.sort_values(by=['Lat'])
df.index = range(df.shape[0])
df = df.iloc[34:,:]
df = df[df.Product_title!='0']
df.index = range(df.shape[0])

In [ ]:
# Download multiple products
# In case you want to download multiple products given tile and image_date a for loop can be used.
# Start date must be in this format: YYYY/MM/DD
for k in range(df.shape[0]):
    key_json = r'/home/antonis/sentinel-2/elegant-tide-219210-8cae76926534.json' # Path to .json key file
    project_id = 'elegant-tide-219210' # Created project id from google platform
    outdir = r'/mnt/data4/antonis/no_coordinates'
    tile = df.tile.iloc[k]
    cloud = 20
    strip = df.Image_date.iloc[k].split("/",2)
    #start = '2016/07/09' , datetime.strptime('2018/01/18',"%Y/%m/%d")
    start = datetime.strptime(strip[2]+str('/')+strip[0]+str('/')+strip[1],"%Y/%m/%d")
    #end = '2016/07/11'
    end = datetime.strptime(strip[2]+str('/')+strip[0]+str('/')+strip[1],"%Y/%m/%d")

    scene_list = query_sentinel(key_json, project_id, start, end, tile, cloud)
    for s in scene_list:
        download_sentinel(s, outdir)

S2B_MSIL1C_20190418T071619_N0207_R006_T39RTM_20190418T095438 ; cloud cover: 4.5135
	 ... *B01.jp2
	 ... *B02.jp2
	 ... *B03.jp2
	 ... *B04.jp2
	 ... *B05.jp2
	 ... *B06.jp2
	 ... *B07.jp2
	 ... *B08.jp2
	 ... *B09.jp2
	 ... *B10.jp2
	 ... *B11.jp2
	 ... *B12.jp2
	 ... *B8A.jp2
	 ... *TCI.jp2
	 ... *PVI.jp2
S2B_MSIL1C_20190418T071619_N0207_R006_T39RTM_20190418T095438 ; cloud cover: 4.5135
	 ... *B01.jp2
	 ... *B02.jp2
	 ... *B03.jp2
	 ... *B04.jp2
	 ... *B05.jp2
	 ... *B06.jp2
	 ... *B07.jp2
	 ... *B08.jp2
	 ... T39RTM_20190418T071619_B08.jp2 FAILED!
	 ... *B09.jp2


In [100]:
# Download one product
# Or download one product by user-defined parameters like tile and start date
key_json = r'/home/antonis/sentinel-2/elegant-tide-219210-8cae76926534.json'
project_id = 'elegant-tide-219210'
outdir = r'/home/antonis/sentinel-2/coordinates'
tile = '37QDD'
cloud = 20
#strip = df.Image_date.iloc[k].split("/",2)
start = datetime.strptime('2019/10/12',"%Y/%m/%d")
#start = datetime.strptime(strip[2]+str('/')+strip[0]+str('/')+strip[1],"%Y/%m/%d")
#end = '2016/07/11'
end = start

scene_list = query_sentinel(key_json, project_id, start, end, tile, cloud)
for s in scene_list:
    download_sentinel(s, outdir)